In [2]:
import random as random
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit
from utils_copy import al_integrity_testt
from old_for_testing.sp_approx import sp_approx
from old_for_testing.utils import *


score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [3]:
def multiple_merge_algo(seqs: list[str], score_matrix: dict, gap_cost: int, verbose=False, return_center_string=False,layout="spring"):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2,verbose=verbose))
    if verbose:
        print("Here comes the distance matrix produced by the alignments: \n")
        print(matrix)
    matrix_for_MST=matrix #copy the matrix, so that we can keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    TODO: #till here I'd say we need no changes!!#
    min_span_edges=find_min_span_edges_testing(matrix_for_MST,verbose=verbose) #Run Kruskal's algorithm on the "pseudomatrix"
    if verbose:
        print("Here comes the pseudomatrix, filled out with with the edges inclued in the MST: \n")
        print(min_span_edges)

    max_indices = np.where(matrix == np.max(matrix)) # Choosing where to start traversal. I want to start at one of the nodes that is the furthest away from any other so start from a side of graph.. hmmmm...
    max_row_index = max_indices[0][0] #just choose one of them.
    if verbose:
        print("Starting key for traversal based on max_row_idex: ")
        print(max_row_index)

    #Put the nodes and the minimum spanning edges into a graph.
    G=fill_graph(min_span_edges,str(int(max_row_index)),layout) #using the max_row_index as the starting key! (an making the graph!)
    alignment_pairs,index_dict=my_traversal_simply(G,str(int(max_row_index)),verbose=verbose) #'traverse' to get alignment_pairs (pairs of sucessors and predecessors) and their position in the MSA to come (index_dict)
    if verbose:
        print("Here come your alignment pairs and the idex dict: \n")
        print(alignment_pairs)
        print(index_dict)

     # Constructing alignment M
    M: list[list[str]] = [[letter] for letter in [*seqs[int(max_row_index)]]] #make structure where evey column in the alignment is represented as a string in a list (in a list)
    cost_list = []
    A_dict={}
    #using the pairings of predecessors and successors in the alignment_pairs dict, align the strings.
    for key,value in alignment_pairs.items():
        if verbose: print("this is the key: "+str(key)+" and this is the value: "+str(value))
        cost = linear_C(gap_cost, score_matrix, seqs[int(value)], seqs[int(key)],verbose=verbose) #the alignment call itself :) 
        if verbose: print("\n now aligning...."+str(seqs[int(key)])+ " and "+ str(seqs[int(value)]))
        cost_list.append(get_cost_2(cost))
        
        # prepare A-matrix for extension
        alignment1_str, alignment2_str = linear_backtrack(seqs[int(value)], seqs[int(key)], cost, score_matrix, gap_cost,verbose=verbose) #backtract to get the alignments!
        str1_nr=value #the predecessor/parent string
        alignment1, alignment2 = [*alignment1_str], [*alignment2_str] #splitting up the alignments into elements to have the right format for the list of lists (M)
        
        A = [list(e) for e in zip(alignment1,alignment2)] #zipping the elements of the two aligned strings together pairwisely
        if ord(key) < ord(value):
            key_for_A_dict=str(key)+"_"+str(value)
        else:
            key_for_A_dict=str(value)+"_"+str(key)
        A_dict[key_for_A_dict]=A
        #pair_al=A_dict[key_for_A_dict]
        if verbose: print("A right now is: "+str(A))
        if verbose: print("M right now: "+str(M))
        # extend
        Mk = extend_alignment_chaos(M,str1_nr, A,index_dict,verbose=verbose) 
        M = Mk
    if verbose:
        print("Here is the alignment in full omg: \n")
        print(M)
    print("this is A_dict: ")
    print(A_dict)
    al_integrity_testt(A,alignment_pairs,index_dict,M,verbose=verbose)
    # ACTUALLY COMPUTE (approximate) COST
    total_cost = compute_cost(M, score_matrix, gap_cost)
    print("Total cost of MSA:"+str(total_cost))
    return total_cost, M, matrix_for_MST,G

SyntaxError: invalid syntax (2203443271.py, line 13)

In [ ]:
def extend_alignment_chaos_multiple(M,str1_nr,A,index_dict,MSA_of_str2, verbose=False): #needs inclusion of str1_nr, to come from the outside.... str1_nr is the name "predecessor" string  
    MA = []
    i = 0
    j = 0
    col_in_M_of_parent_string=int(index_dict[str1_nr]) #getting the column from the index dict that holds the "predecessor" string, to know where it is in the MSA
    while i < len(M) and j < len(A):
        if verbose: print("i:"+str(i)+", j:"+str(j))
        if verbose: print("parent string nr: "+ str(str1_nr))
        if verbose: print("which is in col:"+str(col_in_M_of_parent_string))
        if verbose: print("I compare "+ str(M[i][col_in_M_of_parent_string])+" and "+str(A[j][0]))
        # Case 1:
        if M[i][col_in_M_of_parent_string] == '-' and A[j][0] == '-':
            if verbose: print("I go to case 1 (two gaps)")
            #M[i].append(A[j][1])
            M[i].append(MSA_of_str2[j])#just append the whole damn column!
            MA.append(M[i])
            if verbose: print("Now MA is this: \n "+ str(MA))
            i = i + 1
            j = j + 1

        # Case 2
        elif M[i][col_in_M_of_parent_string] == '-' and A[j][0] != '-':
            if verbose:print("I go to case 2 (gap,char)")
            for char in MSA_of_str2[0]:#append as many gaps as you there are chars (cols) in MSA of str2
                M[i].append('-')
                MA.append(M[i])
            if verbose: print("Now MA is this: \n "+ str(MA))
            i = i + 1
        
        # Case 3:
        elif M[i][col_in_M_of_parent_string] != '-' and A[j][0] == '-':
            if verbose:print("I go to case 3 (char, gap)")
            c = ['-']*len(M[i])
            c.append(MSA_of_str2[j])
            MA.append(c)
            if verbose: print("Now MA is this: \n "+ str(MA))
            j = j + 1
        
        # Case 4:
        elif M[i][col_in_M_of_parent_string] != '-' and A[j][0] != '-':
            if verbose: 
                print("I go to case 4 (char, char)")
            M[i].append(MSA_of_str2[j]) #just put the whole damn col/row from the MSA of str2
            MA.append(M[i])
            if verbose: print("Now MA is this: \n "+ str(MA))
            i = i + 1
            j = j + 1
    #when one of the strings has ended, put in gaps, till the other ends as well. 
    if i < len(M):
        while i < len(M):
            for char in MSA_of_str2[0]:
                M[i].append('-')
                MA.append(M[i])
            i = i + 1
            
    if j < len(A):
        k = len(M[col_in_M_of_parent_string])
        while j < len(A):
            c = ['-']*(k-1)
            c.append(MSA_of_str2[j])
            MA.append(c)
            j = j + 1
    return MA


In [11]:
test_res_mat=np.array([['*', '1', '0', '1'], ['*', '2', '0', '2'],['','3','0','2'], ['', '3', '1', '2']])

In [19]:
def orchestrate_merge(res_matrix, seqs,score_matrix,gapcost,verbose=False): #should also include seqs,score_matrix,gap_cost, verbose=False)
    in_which_MSA_is_it={}
    names= np.unique(res_matrix[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    MSA_list=[[*seq] for seq in seqs] #initalizing it as every string in their own alignment with nobody else :) 
    print(in_which_MSA_is_it)
    for row in res_matrix:
        if row[0]=="*":
            node1 = row[2]  # Extract the first node
            node2 = row[3]  # Extract the second node
            cost=linear_C(gap_cost,score_matrix,seqs[node1],seqs[node2]) #doing the alignment of the two strings :) 
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost,verbose=verbose)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)] #the 'guide' (pairwise alignment) in this iteration!
            #where are these two sequences now? get the MSA and their position in that MSA!
            which_MSA_node1=in_which_MSA_is_it[node1][0]
            which_pos_in_MSA_node1=in_which_MSA_is_it[node1][1]
            which_MSA_node2=in_which_MSA_is_it[node2][0]
            which_pos_in_MSA_node2=in_which_MSA_is_it[node2][1]
            current_MSA_node1=MSA_list[which_MSA_node1][which_pos_in_MSA_node1] #how is the current MSA of node 1 looking??
            current_MSA_node2=MSA_list[which_MSA_node2][which_pos_in_MSA_node2] #how is the current MSA of node 1 looking??
            #now we should compare the A (pairwise alignment) to the current alignment





merge_actually() #TODO: #um actually make this lollll




In [28]:
names=np.unique(test_res_mat[:, 2:])
print(names)
#in_which_MSA_is_itt = {name:[int(name):0 for i, name in enumerate(names)]}
in_which_MSA_is_itt = {name: [int(name),0] for name in names}
print(in_which_MSA_is_itt)

seqss=['agct','ctgg','gtacccacaca']
MSA_list=[[*seq] for seq in seqss]
print(MSA_list)


['0' '1' '2']
{'0': [0, 0], '1': [1, 0], '2': [2, 0]}
[['a', 'g', 'c', 't'], ['c', 't', 'g', 'g'], ['g', 't', 'a', 'c', 'c', 'c', 'a', 'c', 'a', 'c', 'a']]


In [20]:
orchestrate_merge(test_res_mat)

{0: {'0': 0}, 1: {'1': 0}, 2: {'2': 0}}

In [39]:
def make_MSA_position_overview_dict(res_matrix):#initializes the overview dictionary
    in_which_MSA_is_it={}
    names= np.unique(res_matrix[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    print(in_which_MSA_is_it)

make_MSA_position_overview_dict(test_res_mat)

{'0': [0, 0], '1': [1, 0], '2': [2, 0]}


In [45]:
def find_highest_second_element(d, first_element):
    highest_second_element = 0
    for key, value in d.items():
        if value[0] == first_element:
            second_element = value[1]
            if second_element > highest_second_element:
                highest_second_element = second_element
    return highest_second_element

In [48]:
def update_MSA_position_overview_dict(MSA_position_overview_dict,node1,node2): #may be somehow inadequate!
    print(node1,node2)
    node1_origin_MSA=MSA_position_overview_dict[node1][0]
    node2_origin_MSA=MSA_position_overview_dict[node2][0]
    node1_origin_MSA_col=MSA_position_overview_dict[node1][1]
    node2_origin_col=MSA_position_overview_dict[node2][1]
    #now I have the original positions in from the position overview dict!
    nr_we_got_to=find_highest_second_element(MSA_position_overview_dict,MSA_position_overview_dict[node1])+1
    MSA_position_overview_dict[node2]=[node1_origin_MSA,(nr_we_got_to+1)] #update the pos_dict to show that node2 changed MSA and make the col one bigger than the max
    return MSA_position_overview_dict

overview={'0': [0, 0], '1': [1, 0], '2': [2, 0]}
update_MSA_position_overview_dict(overview,'1','2')

1 2


{'0': [0, 0], '1': [1, 0], '2': [1, 2]}

In [5]:
def get_MSA_pos(MSA_position_overview_dict, node):
    MSA_pos=MSA_position_overview_dict[node][0]
    MSA_pos_col=MSA_position_overview_dict[node][1]
    return MSA_pos,MSA_pos_col

get_MSA_pos(overview,'2')

(2, 0)

In [6]:
def identify_merge_case(guide_PA,MSA1,node1,MSA_position_overview_dict,i,j): 
    case=None
    node1_MSA,node1_col=get_MSA_pos(MSA_position_overview_dict,node1)
    if guide_PA[i][0]=='-' and MSA1[j][node1_col]=='-':
        case=1
    elif guide_PA[i][0]!='-' and MSA1[j][node1_col]=='-':
        case=2
    elif guide_PA[i][0]=='-' and MSA1[j][node1_col]!='-':
        case=3
    elif guide_PA[i][0]!='-' and MSA1[j][node1_col]!='-':
        case=4
    return case

fake_MSA1=[['-','a'],['a','g']]
fake_guide_PA=[['-','c'],['-','g']]

identify_merge_case(fake_guide_PA,fake_MSA1,'1',overview,1,1) #sounds about right!!

3

In [35]:
#ok we have the guide_PA, and the MSA1 and MSA2 that the strings are currently in, the MSA_position_overview to keep track of
#in which col a node/string is in. We use this func when guide_PA[i][0]=='-' and MSA1[j][node1_col]=='-'.if they are both at a gap.
#so this is a case when we just wanna put the whole thing of MSA (of that colum), because it looks good!
#assuming that the guide_PA was made to between how the strings were already looking in the MSA's it the columns in the guide PA and
#MSA would be the same enableling a simple enough merge. May be a stupid choice, but let's see.
def handle_case1(MSA1,MSA2,i,j):
    col_new=MSA1[j]+MSA2[i]
    return col_new
#this next case is invoked if guide_PA[i][0]!='-' and MSA1[j][node1_col]=='-'.
#so the guide wants a non-gap. But we are at a gap in the MSA1. So we need to add gaps for every row in the col of MSA2, and put none of MSA2 in there.
def handle_case2(MSA1,MSA2,i,j):
    how_many_cols_in_MSA2=len(MSA2[0])
    gaps_to_append=['-']*how_many_cols_in_MSA2
    col_now=MSA1[j]
    col_new=MSA1[j]+gaps_to_append
    return col_new

#now we are in a situation where guide_PA[i][0]=='-' and MSA1[j][node1_col]!='-'.
#meaning that the guide wants a gap, and but there is a char in the MSA.
#so now we need to add amount of gaps corresponding to the rows in a col of MSA1. And then add the whole col of MSA1.
def handle_case3(MSA1,MSA2,i,j): #this case seems solid!
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+MSA2[i]
    return new_col

#now we are in a situation where guide_PA[i][0]!='-' and MSA1[j][node1_col]!='-'.
#meaning that we are in a place where we just need to put in the whole col from MSA2 into MSA1
def handle_case4(MSA1,MSA2,i,j):
    col_new=MSA1[j]+MSA2[i]
    return col_new


fake_MSA1=[['a','a','g'],['a','g','t']]
fake_MSA2=[['c','-','a'],['-','a','a']]
fake_guide_PA=[['a','c'],['-','g']]
overview={'0': [0, 0], '1': [1, 0], '2': [2, 0]}
case=identify_merge_case(fake_guide_PA,fake_MSA1,'0',overview,0,0)
print(case)
result=handle_case4(fake_MSA1,fake_MSA2,0,0)
print(result)



4
['a', 'a', 'g', 'c', '-', 'a']


In [44]:
#to be used if we have reached the end of the guide MSA, but not the MSA1
def finish_running_through_MSA1(MSA1,MSA2,j,i):
    how_many_cols_in_MSA2=len(MSA2[0])
    gaps_to_add=['-']*how_many_cols_in_MSA2
    new_col=MSA1[j]+gaps_to_add
    return new_col

def finish_running_through_MSA2(MSA1,MSA2,j,i):
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+MSA2[i]
    return new_col

fake_MSA1=[['a','a','g'],['a','g','t'],['a','t','c']]
fake_MSA2=[['c','-','a'],['-','a','a']]

res1=finish_running_through_MSA1(fake_MSA1,fake_MSA2,2,1)
res2=finish_running_through_MSA2(fake_MSA1,fake_MSA2,2,1)

print(res1,res2)

['a', 't', 'c', '-', '-', '-'] ['-', '-', '-', '-', 'a', 'a']


In [ ]:
#this function is to to ONE merge of two strings and their respective MSA that they are stuck in...
def merge_united(guide_PA,MSA_list,in_which_MSA_is_it,node1,node2):
    MSA_of_node1,row_of_node1=get_MSA_pos(in_which_MSA_is_it,node1)
    MSA_of_node2,row_of_node2=get_MSA_pos(in_which_MSA_is_it,node2)
    MSA1=MSA_list[MSA_of_node1]
    MSA2=MSA_list[MSA_of_node2]
    
    j=0
    i=0
    MSA_new=[]
    while j < len(MSA1) and j < len(MSA2): #TODO:#is this even the correct while-statement????
        case=identify_merge_case(guide_PA,MSA1,MSA2,node1,node2,j,i)
        if case==1:
            column=handle_case1(MSA1,MSA2,i,j)
            MSA_new.append(column)
            j+=1
            i+=1
        elif case==2:
            column=handle_case2(MSA1,MSA2,i,j)
            MSA_new.append(column)
            j+=1
        elif case==3:
            column=handle_case3(MSA1,MSA2,i,j)
            MSA_new.append(column)
            i+=1
        elif case==4:
            column=handle_case4(MSA1,MSA2,i,j)
            MSA_new.append(column)
            j+=1
            i+=1

        
        